# Trabajo Final de Graduación
Daniel Núñez Vargas

In [1]:
import importlib
from Funciones import normalize_block_name # Script donde se han incluido variedad de funciones personalizadas
import pandas as pd
import numpy as np
import re

In [2]:
# Se cargan los datos
df = pd.read_csv("../Input/Incidents_2025.csv")
df.head(10)

,the_geom,cartodb_id,the_geom_webmercator,objectid,dc_dist,psa,dispatch_date_time,dispatch_date,dispatch_time,hour,dc_key,location_block,ucr_general,text_general_code,point_x,point_y,lat,lng
0,0101000020E6100000483D5F41A7C952C010D2996F95F7...,1,0101000020110F00004C03D720AEE95FC14B4695FD9186...,29086214,3,2,2025-05-03 03:44:00+00,2025-05-02,23:44:00,23,2.025030e+11,1100 BLOCK S 4TH ST,300,Robbery No Firearm,-75.150833,39.934248,39.934248,-75.150833
1,0101000020E6100000942D829472CB52C00868633E43FB...,2,0101000020110F0000FE7F6C56BAEC5FC10F56395FA58A...,28777819,9,1,2025-04-23 22:18:00+00,2025-04-23,18:18:00,18,2.025090e+11,300 BLOCK MARTIN LUTHER KING DR,300,Robbery No Firearm,-75.178868,39.962990,39.962990,-75.178868
2,0101000020E6100000F0BE9E6D7DCE52C0D06F6D5510FD...,3,0101000020110F00004A86194AE5F15FC1D39EAC59A48C...,29429836,19,3,2025-05-13 01:04:00+00,2025-05-12,21:04:00,21,2.025190e+11,5100 BLOCK LANCASTER AVE,300,Robbery No Firearm,-75.226406,39.977061,39.977061,-75.226406
3,0101000020E6100000A05D1DB1EACB52C048CAED36E6FD...,4,0101000020110F0000FD90065C86ED5FC138217568918D...,28777844,22,4,2025-04-23 22:32:00+00,2025-04-23,18:32:00,18,2.025220e+11,1700 BLOCK N 32ND ST,300,Robbery No Firearm,-75.186199,39.983588,39.983588,-75.186199
4,0101000020E61000002C3AC80797C952C0C8F6274CF300...,5,0101000020110F0000A4AB8C9192E95FC15E540453F390...,28777849,25,4,2025-04-24 04:47:00+00,2025-04-24,00:47:00,0,2.025250e+11,1300 BLOCK W Venango St,300,Robbery No Firearm,-75.149843,40.007425,40.007425,-75.149843
5,0101000020E610000078D60B9155CC52C00609171317FB...,6,0101000020110F0000365F9FE53BEE5FC11CBBED6D748A...,27244917,16,1,2025-02-27 01:16:00+00,2025-02-26,20:16:00,20,2.025160e+11,400 BLOCK N 35TH ST,300,Robbery No Firearm,-75.192723,39.961642,39.961642,-75.192723
6,0101000020E6100000644F029B73C852C018B9196EC003...,7,0101000020110F0000E1C1D58DA3E75FC1E1C6609C0E94...,28403857,35,2,2025-04-13 07:15:00+00,2025-04-13,03:15:00,3,2.025350e+11,5100 BLOCK N 5TH ST,300,Robbery No Firearm,-75.132056,40.029310,40.029310,-75.132056
7,0101000020E6100000A430E247C4C952C058B25A68C002...,8,0101000020110F0000B0BA4F6EDFE95FC1909951A8F292...,28403859,39,2,2025-04-12 13:37:00+00,2025-04-12,09:37:00,9,2.025390e+11,4400 BLOCK N BANCROFT ST,300,Robbery No Firearm,-75.152605,40.021497,40.021497,-75.152605
8,0101000020E6100000F08D342DDBCA52C090D112E24C08...,9,0101000020110F00005BD6D829B9EB5FC1F842D8821A99...,28777858,14,1,2025-04-24 00:15:00+00,2025-04-23,20:15:00,20,2.025140e+11,7700 BLOCK MANSFIELD AV,400,Aggravated Assault No Firearm,-75.169627,40.064846,40.064846,-75.169627
9,0101000020E610000054BFD2F9F0CB52C0F88FBE49D3FE...,10,0101000020110F00008136940891ED5FC11FE5B732988E...,28777859,22,2,2025-04-23 14:40:00+00,2025-04-23,10:40:00,10,2.025220e+11,2200 BLOCK N 33RD ST,400,Aggravated Assault No Firearm,-75.186583,39.990823,39.990823,-75.186583


## Descripción de los datos

Las variables que componen estos datos son:

- **the_geom**: Geometría espacial en formato WKT (Well-Known Text) - representa la ubicación geográfica exacta del incidente
- **cartodb_id**: Identificador único secuencial asignado por CartoDB para cada registro
- **the_geom_webmercator**: Geometría espacial en proyección Web Mercator - formato optimizado para mapas web
- **objectid**: Identificador único del objeto en la base de datos original del sistema policial
- **dc_dist**: Código del distrito policial (1-25) - división administrativa de la policía de Filadelfia
- **psa**: Área de Servicio Policial (Police Service Area) - subdivisión más pequeña dentro de cada distrito
- **dispatch_date_time**: Fecha y hora completa cuando se despachó la llamada de emergencia
- **dispatch_date**: Solo la fecha cuando se despachó la llamada (formato YYYY-MM-DD)
- **dispatch_time**: Solo la hora cuando se despachó la llamada (formato HH:MM:SS)
- **hour**: Hora del día extraída de dispatch_time (0-23) - útil para análisis temporal
- **dc_key**: Clave única del incidente en el sistema de despacho - combina fecha y número secuencial
- **location_block**: Descripción textual de la cuadra donde ocurrió el incidente (ej: "1100 BLOCK S 4TH ST")
- **ucr_general**: Código numérico del Uniform Crime Reporting (UCR) - sistema estándar de clasificación de delitos
- **text_general_code**: Descripción textual del tipo de delito según clasificación UCR
- **point_x**: Coordenada X (longitud) en sistema de coordenadas de Pensilvania
- **point_y**: Coordenada Y (latitud) en sistema de coordenadas de Pensilvania
- **lat**: Latitud en grados decimales (WGS84) - coordenada geográfica estándar
- **lng**: Longitud en grados decimales (WGS84) - coordenada geográfica estándar

Hay varias variables que para efectos del análisis no resultan imporantes, como los identificadores únicos **cartodb_id**, **objectid** y **objectid**. Además, hay algunas otras que explican lo mismo, como lo son las **dispatch_date_time** con **dispatch_date** y **hour**. Asimismo, **point_x** y **point_y** que dan las coordenadas de acuerdo al formato local. Otras variables que a priori se pueden omitir serán **the_geom** y **the_geom_webmercartor**, ya que se le dará prioridad al uso de latitud y longitud en formato estándar (WGS84) para identificación geoespacial de patrones.

Por el momento, la lista de variables que se utilizarán serán:

In [3]:
df_filter = df[['lat', 'lng', 'hour', 'dispatch_date', 'dc_dist', 'psa', 'ucr_general', 'text_general_code', 'location_block']]
df_filter.head(5)

,lat,lng,hour,dispatch_date,dc_dist,psa,ucr_general,text_general_code,location_block
0,39.934248,-75.150833,23,2025-05-02,3,2,300,Robbery No Firearm,1100 BLOCK S 4TH ST
1,39.962990,-75.178868,18,2025-04-23,9,1,300,Robbery No Firearm,300 BLOCK MARTIN LUTHER KING DR
2,39.977061,-75.226406,21,2025-05-12,19,3,300,Robbery No Firearm,5100 BLOCK LANCASTER AVE
3,39.983588,-75.186199,18,2025-04-23,22,4,300,Robbery No Firearm,1700 BLOCK N 32ND ST
4,40.007425,-75.149843,0,2025-04-24,25,4,300,Robbery No Firearm,1300 BLOCK W Venango St


Dado que las coordenadas geográficas serán variables claves dentro del modelo, lo primero será la forma de tratarlas en caso en donde no hayan datos. Lo primero que se nota es que se identificaron 2922 registros que no presentan información sobre latitud o longitud. El primer acercamiento que tendremos será identificar estas observaciones y analizar sus similitudes con observaciones que sí tengan información sobre sus coordenadas.

In [4]:
# Se filtran aquellas observaciones que no tienen algunas de las coordenadas
df_nan = df_filter[(df_filter['lat'].isna()) | (df_filter['lng'].isna())]
# Ahora se extren las ubicaciones de manera única
location_blocks_nan = df_nan['location_block'].unique()

df_temp = df_filter[df_filter['location_block'].str.contains(
    r'1900\s+block\s+s\s+christopher\s+columbus', 
    case=False, na=False
)]

print(df_temp[['location_block', 'lat', 'lng']].head(2))

                             location_block        lat        lng
130   1900 BLOCK S CHRISTOPHER COLUMBUS BLV        NaN        NaN
194  1900 BLOCK S CHRISTOPHER COLUMBUS BLVD  39.922174 -75.142375


Notamos algo interesante ya que para un registro sí se tienen coordenadas de ese bloque mientras que para otro no, siendo la única diferencia entre registros la palabra "BLVD". Este compartamiento se repite para miles de observaciones, por lo que antes de imputar la latitud y la longitud de todos registros que no tienen valor, el primer paso será depurar la variable location_block, ya que esta nos permitirá realizar la imputación basándonos en el bloque en el ocurrió el delito.

Ahora se aplica dicha función a todos los registros:

In [5]:
df_filter['location_block_normalized'] = df_filter['location_block'].apply(normalize_block_name)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10224\3097380153.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter['location_block_normalized'] = df_filter['location_block'].apply(normalize_block_name)


Ahora examinamos la cardinalidad de las variables latitud y longitud, por bloque: 

In [6]:
coordinates_info_by_block = (df_filter.groupby(['location_block_normalized']).agg(
    unique_latitudes=('lat', 'nunique'),
    unique_longitudes=('lng', 'nunique')
)).sort_values(by = 'unique_latitudes', ascending=False)
print(coordinates_info_by_block.head(5))

                           unique_latitudes  unique_longitudes
location_block_normalized                                     
4000 BLOCK LANCASTER AVE                 38                 38
GERMANTOWN AVE                           33                 34
N BROAD ST                               33                 33
FRANKFORD AVE                            31                 31
3100 BLOCK KENSINGTON AVE                30                 30


Como se observa en el cuadro anterior, un bloque puede tener más de una latitud y longitud. Esto sucede porque los bloques fueron redondeados al bloque de cien más cercano. Tomemos el caso que presenta mayor cardinalidad para efectos de la explicación, el cual es **4000 BLOCK LANCASTER AVE**. 

Si tomamos todas las longitudes y latitudes para este bloque, y se calcula el mínimo y el máximo nos damos cuenta que realmente todos lo registros en latitud están entre los 39.9631° y los 39.9652°. De manera análoga para longitud, ubicando todos los crímenes entre los -75.2055° y los -75.2026°.

In [7]:
df_temp = df_filter[df_filter['location_block_normalized'] == '4000 BLOCK LANCASTER AVE'][['location_block_normalized', 'lat', 'lng']]
print(df_temp['lat'].quantile([0,1]).round(4))
print(df_temp['lng'].quantile([0,1]).round(4))

0.0    39.9631
1.0    39.9652
Name: lat, dtype: float64
0.0   -75.2055
1.0   -75.2026
Name: lng, dtype: float64


Aunque existan diferentes longitudes y latitudes para un mismo bloque, este hallazgo lo que nos indica es que la diferencia en términos de grados es marginal por lo que para efectos de hacer imputación para aquellos registros que no tienen coordenadas, se tomará el primer set de coordenadas encontradas para dicho bloque. 

In [8]:
# Extraemos los registros que NO tienen missing en sus coordenadas
df_non_nan_unique = (
    df_filter[~(df_filter['lat'].isna() | df_filter['lng'].isna())]
    .sort_values('dispatch_date')  # opcional: para priorizar por fecha
    .drop_duplicates(subset='location_block_normalized', keep='first')
    [['location_block_normalized', 'lat', 'lng']]
)
# Se filtran aquellas observaciones que SÍ tienen missing en sus coordenadas
df_nan_clean = df_filter[(df_filter['lat'].isna()) | (df_filter['lng'].isna())].drop(['lat', 'lng'], axis=1)
# Se procede a hacer el leftjoin entre ambos datasets
df_nan_join = df_nan_clean.merge(
    df_non_nan_unique,
    on='location_block_normalized',
    how='left',
)

Esto nos corrige bastantes registros como observamos anteriormente:

In [9]:
df_nan_join[['location_block_normalized', 'lat', 'lng']].head(10)

,location_block_normalized,lat,lng
0,1900 BLOCK S CHRISTOPHER COLUMBUS BLVD,39.922174,-75.142375
1,GERMANTOWN AVE,40.023927,-75.159552
2,1600 BLOCK S CHRISTOPHER COLUMBUS BLVD,39.925683,-75.143078
3,1600 BLOCK S CHRISTOPHER COLUMBUS BLVD,39.925683,-75.143078
4,7900 BLOCK E ROOSEVELT BLVD,40.055555,-75.049197
5,12000 BLOCK E ROOSEVELT BLVD,40.105865,-75.003306
6,2100 BLOCK MAGEE AVE,40.036936,-75.066568
7,E CHELTENHAM AVE,40.017513,-75.065115
8,1200 BLOCK S FRONT ST,39.931280,-75.146197
9,2600 BLOCK W BERKS ST,39.984736,-75.176187


Tratando los missing values de esta forma, hemos podido recuperar información de 2216 observaciones. Analicemos qué puede estar pasando con las otras 706. 

Hay una cantidad de registros que lo indican es la intersección entre dos bloques, por eso no llevan el número de bloque al inicio. 

In [10]:
df_still_nan = df_nan_join[df_nan_join['lat'].isna() | df_nan_join['lng'].isna()]

In [11]:
df_non_nan = df_filter[~(df_filter['lat'].isna() | df_filter['lng'].isna())]
df_nan_filled = df_nan_join[['lat', 'lng', 'hour', 'dispatch_date', 'dc_dist', 'psa', 'ucr_general', 'text_general_code', 'location_block', 'location_block_normalized']]